In [1]:
import tensorflow as tf
import keras
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model, Model
from keras.layers import *
import os
import sys
import numpy as np
from score_dataset_helpers import *
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/data_dual_inputs/'
train_dir = base_dir + 'train'
validation_dir = base_dir + 'validation'
test_dir = base_dir + 'test'

## Define model

In [114]:
f1, f2, f3 = 8,16,32
k_size = (3,3)

pics_input = Input(shape = (75, 250, 3))
pics_c1 = Conv2D(filters = f1, kernel_size = k_size, activation = "relu")(pics_input)
pics_m1 = MaxPooling2D(k_size)(pics_c1)
pics_c2 = Conv2D(filters = f2, kernel_size = k_size, activation = "relu")(pics_m1)
pics_m2 = MaxPooling2D(k_size)(pics_c2)
pics_c3 = Conv2D(filters = f3, kernel_size = k_size, activation = "relu")(pics_m2)
pics_m3 = MaxPooling2D((2,2))(pics_c3)
pics_f = Flatten()(pics_m3)
pics_d = Dense(265, activation = "softmax")(pics_f)

nums_input = Input(shape = ([1]))
nums_features = Dense(10, activation = "softmax")(nums_input)

conc_layer = concatenate([pics_d, nums_features])
dense_1 = Dense(50, activation = "softmax")(conc_layer)
dense_2 = Dense(1, activation = "softmax")(dense_1)

model = Model(inputs = [pics_input, nums_input], outputs = dense_2)
model.compile(loss = "binary_crossentropy", optimizer = "sgd", metrics = ["accuracy"])

In [115]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 75, 250, 3)   0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 73, 248, 8)   224         input_21[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_31 (MaxPooling2D) (None, 24, 82, 8)    0           conv2d_31[0][0]                  
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 22, 80, 16)   1168        max_pooling2d_31[0][0]           
__________________________________________________________________________________________________
max_poolin

## Create vectors of scores from pictures

In [133]:
train_scores = train_dir + "/score"
train_pics = train_dir + "/pics"

#valid_scores_up = validation_dir + "/up/score"

#test_scores_up = test_dir + "/up/score"

In [134]:
train_names_scores = [f for f in os.listdir(train_scores)]
train_names_pics = [f for f in os.listdir(train_pics)]

#valid_names_up = [f for f in os.listdir(valid_scores_up)]

#test_names_up = [f for f in os.listdir(test_scores_up)]

In [135]:
print(str(len(train_names_scores)) + " training files for scores")
print(str(len(train_names_pics)) + " training files for pictures")

#print(str(len(valid_names_up)) + " validation files up")

#print(str(len(test_names_up)) + " test files up")

517 training files for scores
517 training files for pictures


In [136]:
train_dataset_score = create_dataset("score", train_names_scores, base_dir + "train/")
#validation_dataset = create_dataset("validation/score", valid_names)
#test_dataset = create_dataset("test/score", test_names)

In [137]:
def get_result(result):
    resultstr = ''
    for i in range(5):
        resultstr += str(np.argmax(result[i]))
    return resultstr

In [138]:
score_model = load_model("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/models/score_v2.h5")

In [139]:
#Create vectors for scores and for labels
score_vec = []
for i in range(len(train_names_scores)):
    y_pred = score_model.predict(train_dataset_score[i].reshape(1, 50, 160, 1))
    score_vec.append(get_result(y_pred))

label_vec = ['up' in x for x in train_names_pics]
label_vec = np.array(label_vec)
label_array = label_vec.astype(int)

In [140]:
print_dat = pd.DataFrame({'score' : score_vec, 'label' : label_vec, 'orig_name' : train_names_scores})
print_dat

,label,orig_name,score
0,False,right_1.png,00008
1,False,right_10.png,00073
2,False,right_100.png,00172
3,False,right_101.png,00185
4,False,right_102.png,00190
5,False,right_105.png,00200
6,False,right_106.png,00200
7,False,right_107.png,00226
8,False,right_108.png,00238
9,False,right_109.png,00243


In [141]:
x = load_img(train_pics + "/" + train_names_pics[1], target_size = (75, 250))
x = img_to_array(x)
x.shape


(75, 250, 3)

In [142]:
pic_array = np.zeros((len(train_names_pics), 75, 250, 3))

for i in range(len(train_names_pics)):
    x = load_img(train_pics + '/' + train_names_pics[i], target_size = (75, 250))
    x = img_to_array(x)
    pic_array[i] = x
    
    

In [143]:
pic_array.shape

(517, 75, 250, 3)

In [144]:
pic_array = np.array(pic_array)
score_array = np.array(score_vec)

In [145]:
pic_array.shape

(517, 75, 250, 3)

In [146]:
score_array.shape

(517,)

In [147]:
label_array.shape

(517,)

In [148]:
model.fit([pic_array, score_array], label_array, epochs = 3, batch_size = 50)

Epoch 1/3
517/517 [==============================] - 3s 5ms/step - loss: 10.1143 - acc: 0.3656
Epoch 2/3
517/517 [==============================] - 3s 5ms/step - loss: 10.1143 - acc: 0.3656
Epoch 3/3
517/517 [==============================] - 3s 5ms/step - loss: 10.1143 - acc: 0.3656


In [ ]:



#TODO: generate data, delete white pictures in both images